# Inferencia de tipos

En lenguajes como C, el programador debe dar los tipos explícitamente. Por ejemplo, la siguiente función toma dos `double` y devuelve otro:

```c
double f(double x, double y) {
    return 2*x + y;
}
```

Los tipos de las variables se conocen porque el programador los especifica obligatoriamente. De este modo el compilador puede generar código que haga la operación de manera eficiente.

Por el contrario, en lenguajes interpretados como Python, los tipos se chequean en tiempo de ejecución. Por ejemplo, tomemos el siguiente bloque de código:

```python
x = 1
y = 2
2*x + y
```

En la tercera línea, el intérprete de Python chequea el tipo de los objetos, y los usa para calcular el resultado. Esto genera dos fuentes de overhead:

 1. El chequeo de tipos
 2. La asignación de memoria dinámica, ya que es imposible saber cuánta memoria toma un valor, ni si seguirá tomando la misma cantidad de memoria en el futuro.

La solución de Julia es un híbrido. El código en Julia se ve como el de Python:

```julia
x = 1
y = 2
2*x + y
```

Sin embargo, antes de la compilación, Julia ejecuta un algoritmo de inferencia de tipos. Dado que `x` es un `Int`, e `y` es un `Int`, puede inferir que `2*x+y` también es un `Int`, para así compilar el método más eficiente para realizar la operación posteriormente. 

Para escribir código rápido en Julia, es importante que el algoritmo de inferencia pueda inferir los tipos correctamente y que éstos sean concretos, lo que se denomina "estabilidad con respecto al tipo". Esta propiedad tiene dos ventajas que impactan en la performance: 

1. El código se compila a instrucciones rápidas `GOTO`
2. Se conoce el espacio que ocupan las variables en memoria en tiempo de compilación

El segundo punto implica que la memoria puede asignarse en el *stack* (la sección de la memoria de acceso prácticamente inmediato)

Por el contrario, el código inestable corre con las siguientes desventajas: 
1. Cada llamado a una función debe recorrer la lista de métodos, buscando el que corresponde a los tipos de los argumentos
2. No se conoce el tamaño en memoria de las variables en tiempo de compilación, por lo que se les asigna punteros en el *heap*, donde acceso es mucho más lento. 

Veamos ejemplos de esto en Julia. Definimos, por ejemplo, una suma estable con respecto al tipo:

In [9]:
suma_estable(x,y) = x + y

suma_estable (generic function with 1 method)

Inspeccionemos el código LLVM (representación intermedia) al que se compila la suma cuando se llama con dos `Int`:

In [10]:
@code_llvm suma_estable(1,2)

;  @ In[9]:1 within `suma_estable`
define i64 @julia_suma_estable_2022(i64 signext %0, i64 signext %1) #0 {
top:
; ┌ @ int.jl:87 within `+`
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


Ahora, con dos `Float64`:

In [25]:
@code_llvm suma_estable(1.0,2.0)

;  @ In[24]:1 within `suma_estable`
define double @julia_suma_estable_3176(double %0, double %1) #0 {
top:
; ┌ @ float.jl:408 within `+`
   %2 = fadd double %0, %1
; └
  ret double %2
}


¿Qué pasa en el caso mixto?

In [26]:
@code_llvm suma_estable(1, 2.0)

;  @ In[24]:1 within `suma_estable`
define double @julia_suma_estable_3180(i64 signext %0, double %1) #0 {
top:
; ┌ @ promotion.jl:410 within `+`
; │┌ @ promotion.jl:381 within `promote`
; ││┌ @ promotion.jl:358 within `_promote`
; │││┌ @ number.jl:7 within `convert`
; ││││┌ @ float.jl:159 within `Float64`
       %2 = sitofp i64 %0 to double
; │└└└└
; │ @ promotion.jl:410 within `+` @ float.jl:408
   %3 = fadd double %2, %1
; └
  ret double %3
}


Podemos usar `@code_warntype` para inspeccionar si el código es estable con respecto al tipo:

In [28]:
@code_warntype suma_estable(1.0,2.0)

MethodInstance for suma_estable(::Float64, ::Float64)
  from suma_estable(x, y) @ Main In[24]:1
Arguments
  #self#::Core.Const(suma_estable)
  x::Float64
  y::Float64
Body::Float64
1 ─ %1 = (x + y)::Float64
└──      return %1



Definamos ahora una función inestable con respecto al tipo:

In [29]:
function suma_inestable(x,y)
    out = x + y
    rand(Bool) ? out : Float64(out)
end

suma_inestable (generic function with 1 method)

In [30]:
@code_warntype suma_inestable(1,2)

MethodInstance for suma_inestable(::Int64, ::Int64)
  from suma_inestable(x, y) @ Main In[29]:1
Arguments
  #self#::Core.Const(suma_inestable)
  x::Int64
  y::Int64
Locals
  out::Int64
Body::Union{Float64, Int64}
1 ─      (out = x + y)
│   %2 = Main.rand(Main.Bool)::Bool
└──      goto #3 if not %2
2 ─      return out
3 ─ %5 = Main.Float64(out)::Float64
└──      return %5



Un patrón más común que puede llevar a inestabilidad con respecto al tipo es el siguiente (tomado de la documentación de Julia):

In [11]:
pos(x) = x < 0 ? 0 : x

pos (generic function with 1 method)

In [12]:
@code_warntype pos(1.0)

MethodInstance for pos(::Float64)
  from pos(x) @ Main In[11]:1
Arguments
  #self#::Core.Const(pos)
  x::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x



La solución estable sería la siguiente:

In [13]:
pos(x) = x < 0 ? zero(x) : x

pos (generic function with 1 method)

In [14]:
@code_warntype pos(1.0)

MethodInstance for pos(::Float64)
  from pos(x) @ Main In[13]:1
Arguments
  #self#::Core.Const(pos)
  x::Float64
Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x



### Funciones barrera

Tomemos el siguiente ejemplo de una función inestable de la documentación de Julia:

In [8]:
function strange_twos(n)
    a = Vector{rand(Bool) ? Int64 : Float64}(undef, n)
    for i = 1:n
        a[i] = 2
    end
    return a
end

strange_twos (generic function with 1 method)

In [18]:
strange_twos(3)

3-element Vector{Float64}:
 2.0
 2.0
 2.0

In [19]:
@code_warntype strange_twos(3)

MethodInstance for strange_twos(::Int64)
  from strange_twos(n) @ Main In[8]:1
Arguments
  #self#::Core.Const(strange_twos)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  a::Vector
  i::Int64
  @_6::Union{Type{Float64}, Type{Int64}}
Body::Vector
1 ─       Core.NewvarNode(:(@_3))
│         Core.NewvarNode(:(a))
│   %3  = Main.Vector::Core.Const(Vector)
│   %4  = Main.rand(Main.Bool)::Bool
└──       goto #3 if not %4
2 ─       (@_6 = Main.Int64)
└──       goto #4
3 ─       (@_6 = Main.Float64)
4 ┄ %9  = @_6::Union{Type{Float64}, Type{Int64}}
│   %10 = Core.apply_type(%3, %9)::Type{Vector{_A}} where _A
│   %11 = Main.undef::Core.Const(UndefInitializer())
│         (a = (%10)(%11, n))
│   %13 = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%13))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #7 if not %16
5 ┄ %18 = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%18, 1))
│   %20

La inferencia de tipos ocurre en las fronteras entre funciones. Por lo tanto, introducir un llamado a una *función barrera* aliviana el problema, dando un nuevo punto de referencia para la inferencia de tipos.

In [21]:
function fill_twos!(a)
    for i = eachindex(a)
        a[i] = 2
    end
end

function strange_twos(n)
    a = Vector{rand(Bool) ? Int64 : Float64}(undef, n)
    fill_twos!(a)
    return a
end

strange_twos (generic function with 1 method)

Esta segunda versión es más rápida que la anterior, porque la función `fill_twos!` puede recompilarse para distintos tipos de `a`.